In [1]:
import models.models as md
import datasets.datasets as ds
from constants import *
from torchsummary import summary
import torch
import torch.nn as nn
import skorch
from skorch.helper import predefined_split

# Модуль models 

Модуль models на данный момент содержит PyTorch имплементацию следующих моделей:

## AlexNet

[Alex Krizhevsky, Ilya Sutskever, and Geoffrey E. Hinton. 2017. ImageNet classification with deep convolutional neural networks. <i>Commun. ACM</i> 60, 6 (June 2017), 84–90. DOI:https://doi.org/10.1145/3065386]

Архитектура немного переделана, чтобы соответствовать нашей специфике:<br>
1. Количество входных каналов первого сверточного слоя уменьшено с 3 до 1
2. Изменён последний свёрточный слой, его ядро увеличено с (3, 3) до (4, 4)
3. Изменён слой AdaptiveAvgPool2d с (6, 6) до (5, 5)
4. Изменён классификатор, теперь это четыре полносвязных слоя: 6400, 2048, 512, 512, <число классов>
5. Добавлен Dropout() после первого полносвязного слоя

In [2]:
model = md.alexnet(num_classes=4)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # PyTorch v0.4.0
device = torch.device('cpu')
model = model.to(device)
print(model)
summary(model, (1, 224, 224), device='cpu')

AlexNet(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(5, 5))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=6400, out_features=2048, bias=True)
 

In [3]:
iemocap_train = ds.IemocapDataset(
    PATH_TO_PICKLE, IEMOCAP_PATH_TO_WAVS, IEMOCAP_PATH_TO_EGEMAPS, IEMOCAP_PATH_FOR_PARSER,
    base_name='IEMOCAP', label_type='four', spectrogram_shape=224, preprocessing=False, 
    augmentation=False, padding='repeat', mode='train', tasks=['emotion']
)
iemocap_test = ds.IemocapDataset(
    PATH_TO_PICKLE, IEMOCAP_PATH_TO_WAVS, IEMOCAP_PATH_TO_EGEMAPS, IEMOCAP_PATH_FOR_PARSER,
    base_name='IEMOCAP', label_type='four', spectrogram_shape=224, preprocessing=False, 
    augmentation=False, padding='repeat', mode='test', tasks=['emotion']
)
y = [y for X, y in iter(iemocap_train)]
# print(len(set(y)))

============= INITIALIZING DATASET IEMOCAP_four_prep-false_train ===============
=========================== SUCCESS! ====================================
============= INITIALIZING DATASET IEMOCAP_four_prep-false_test ===============
=========================== SUCCESS! ====================================


In [4]:
model = md.alexnet(num_classes=4)
net = skorch.net.NeuralNet(
    model, criterion=nn.CrossEntropyLoss, optimizer=torch.optim.Adam, dataset=iemocap_train,
    lr=0.01, max_epochs=10, batch_size=128, train_split=predefined_split(iemocap_test), 
    callbacks=None, verbose=1, device=device
)

In [ ]:
data, target = iemocap_test[0]
print(target)

In [ ]:
print(data.shape)

In [ ]:
net.fit(iemocap_train)